## Introduction

MFF is a Python package developed for generating reconciled forecasts with exogenous variables. This Jupyter notebook will provide a walkthrough of the main functionality of this package and key classes for users to customize their experience.

The program is comprised of one high-level function MFF. MFF uses three steps to generate forecasts:
1. Parsing constraints
2. Estimating forecasts and residual covariance function
3. Reconciliation

In this notebook, we will cover how to use the MFF function and the individual functionalities for further customization

# Data and input formatting

In [2]:
import os
os.chdir(r'C:\Users\dbilgin\OneDrive - International Monetary Fund (PRD)\prototype\mff')

We load in synthetic data provided for convenience. The data must conform to the following requirements:
- The DataFrame index is the lowest frequency of the data and is either int or pd.PeriodIndex (pd.Datetime cannot be handled).
- The columns are a MultiIndex containing the following levels: variable, freq, and subperiod (e.g., with quarterly data, we might include ('GDP', 'Q', 2))
- There are no NAs in the historical data to be used for estimation

In [3]:
from mff.utils import load_synthetic_data
data, constraints = load_synthetic_data()

print('Historical data sample: \n\n', data.head())
print('\n\n Forecast data: \n\n', data.tail(11))
print('\n\n Constraint list: \n\n', constraints)

Historical data sample: 

 variable     CA     TA    IA    FX   GDP
freq          A      A     A     A     A
subperiod     1      1     1     1     1
year                                    
1994       4.19   4.51 -0.32  0.94  6.21
1995       2.69   2.38  0.30  1.01  7.87
1996      -9.67 -10.22  0.55  0.98  6.94
1997      -8.80  -8.99  0.19  0.90  4.44
1998      -9.57 -10.26  0.69  0.86  4.08


 Forecast data: 

 variable     CA    TA    IA    FX   GDP
freq          A     A     A     A     A
subperiod     1     1     1     1     1
year                                   
2020       0.55  2.11 -1.55  1.14 -3.34
2021      -2.46  0.03 -2.49  1.18  4.79
2022      -2.73 -0.27   NaN  1.05  4.51
2023      -2.98 -0.55   NaN  1.10  4.26
2024        NaN   NaN   NaN  1.10  4.04
2025        NaN   NaN   NaN  1.10  3.86
2026        NaN   NaN   NaN  1.10  3.72
2027      -0.30   NaN   NaN  1.10  3.60
2028        NaN   NaN   NaN  1.10  3.49
2029        NaN   NaN   NaN  1.10  3.39
2030        NaN   NaN  

# Estimation

## High-level MFF class

For user convenience, we provide a default forecaster based on Ando (2024). This forecaster generates the following models in sktime for each variable:
- Naive forecaster/random walk
- VARX estimated via OLS, elastic net and PCA

It then selects the model with the best out-of-sample forecast performance for each variable. Thus, each variables can be forecasted using different models.

We can initialize this forecaster by using the get_default_forecaster(n_lags), where n_lags represnts the number of lags to be included in the estimation.

In [4]:
from mff.default_forecaster import get_default_forecaster
forecaster = get_default_forecaster(4)

C:\ProgramData\Python3\Lib\site-packages\sktime\forecasting\compose\_reduce.py:1917: UserWarning: DirectReductionForecaster is experimental, and interfaces may change. user feedback is appreciated in issue #3224 here: https://github.com/sktime/sktime/issues/3224
  warn(


In [5]:
from mff.mff import MFF
lam = 1e3  # smoothing paramter
n_resid = 2  # number of residuals with which to calculate covariance matrix
cov_matrix_calc = 'monotone_diagonal'  # covariance matrix estimator

mff = MFF(data, constraints, lam=lam, forecaster=forecaster, n_resid=n_resid, cov_calc_method=cov_matrix_calc)
mff.fit()

C:\ProgramData\Python3\Lib\site-packages\sktime\forecasting\compose\_reduce.py:1917: UserWarning: DirectReductionForecaster is experimental, and interfaces may change. user feedback is appreciated in issue #3224 here: https://github.com/sktime/sktime/issues/3224
  warn(


Generating constraints
Generating forecasts
Reconciling forecasts


C:\ProgramData\Python3\Lib\site-packages\sktime\forecasting\compose\_reduce.py:1917: UserWarning: DirectReductionForecaster is experimental, and interfaces may change. user feedback is appreciated in issue #3224 here: https://github.com/sktime/sktime/issues/3224
  warn(
C:\ProgramData\Python3\Lib\site-packages\sktime\forecasting\compose\_reduce.py:1917: UserWarning: DirectReductionForecaster is experimental, and interfaces may change. user feedback is appreciated in issue #3224 here: https://github.com/sktime/sktime/issues/3224
  warn(
C:\ProgramData\Python3\Lib\site-packages\sktime\forecasting\compose\_reduce.py:1917: UserWarning: DirectReductionForecaster is experimental, and interfaces may change. user feedback is appreciated in issue #3224 here: https://github.com/sktime/sktime/issues/3224
  warn(
C:\ProgramData\Python3\Lib\site-packages\sktime\forecasting\compose\_reduce.py:1917: UserWarning: DirectReductionForecaster is experimental, and interfaces may change. user feedback is ap

Once the model is fit, we can recover the unreconciled and reconciled forecasts as attributes of the MFF model. 

In [6]:
print('\n\n Unreconciled forecast: \n\n', mff.y_unreconciled.tail(11))
print('\n\n Reconciled forecast: \n\n', mff.y_reconciled.tail(11))



 Unreconciled forecast: 

 variable         CA        TA        IA    FX   GDP
freq              A         A         A     A     A
subperiod         1         1         1     1     1
year                                               
2020       0.550000  2.110000 -1.550000  1.14 -3.34
2021      -2.460000  0.030000 -2.490000  1.18  4.79
2022      -2.730000 -0.270000 -2.640655  1.05  4.51
2023      -2.980000 -0.550000 -2.213920  1.10  4.26
2024      -3.703929 -1.430714 -2.316152  1.10  4.04
2025      -3.703929 -1.299639 -2.273214  1.10  3.86
2026      -3.703929 -1.220843 -2.273214  1.10  3.72
2027      -3.703929 -1.430714 -2.829761  1.10  3.60
2028      -3.703929 -1.430714 -2.273214  1.10  3.49
2029      -4.096153 -1.809462 -2.273214  1.10  3.39
2030      -3.983858 -1.430714 -2.758412  1.10  3.30


 Reconciled forecast: 

 variable         CA    FX   GDP        IA        TA
freq              A     A     A         A         A
subperiod         1     1     1         1         1
year    

We can also access the underlying specification and estimation attributes for the reconciled and unreconciled forecasts as attributes of an MFF object:
- Constraints matrix (C, d)
- Unconditional forecaster (uncondiitonal_forecaster) and covariance matrix (W)
- Reconciler (reconciler)

.fit() estimates each of these in the above order, but they can also be fit and accessed individually.

In [7]:
# evaluate constraints
mff.parse_constraints()
mff.C, mff.d

# generate forecasts
mff.fit_unconditional_forecaster()
mff.unconditional_forecaster

# fit reconciler
mff.fit_reconciler()
mff.reconciler

# Lower-level classes

## Constraint parser

Constraints are passed in to the parser via a list of strings and a dataframe capturing the forecast dataframe elements and converted to a matrix form. 

The parser will handle wildcard values, so not all constraints must be explicitly specified. Instead, '?' is used as a wildcard value by default. The wildcard will expand along the omitted dimension following the format (variable)\_(year)(freq)(subperiod). For some examples:
- GDP expenditure accounting identity: 'GDP\_? - C\_? - I\_? - G\_? - NX\_?' will expand along all time dimensions provided in the dataframe (i.e., GDP_2021 through GDP_2030).
- Quarterly-to-annual aggregation:'1/4 * (GDP_?Q1 + GDP_?Q2 + GDP_?Q3 + GDP_?Q4) - GDP_?' will expand GDP along the year dimension to restrict annual values to equal the quarterly average. Alternatively, '1/4 * (?Q1 + ?Q2 + ?Q3 + ?Q4) - ?' will perform this aggregation for all variables and years.

In [8]:
from mff.step0_parse_constraints import generate_constraints

C, d = generate_constraints(data,  constraints)
print('\n\n Constraint list: \n\n', constraints)
print('\n\n Constraint matrix: \n\n', C)
print('\n\n Constant matrix: \n\n', d)



 Constraint list: 

 ['CA? - TA? - IA?']


 Constraint matrix: 

 variable                      CA   FX  GDP   IA   TA   CA   FX  GDP   IA   TA  \
year                        2020 2020 2020 2020 2020 2021 2021 2021 2021 2021   
freq                           A    A    A    A    A    A    A    A    A    A   
subperiod                      1    1    1    1    1    1    1    1    1    1   
CA_2020 - 0.55               1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
FX_2020 - 1.14               0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
GDP_2020 - -3.34             0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
IA_2020 - -1.55              0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0   
TA_2020 - 2.11               0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0   
CA_2021 - -2.46              0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0   
FX_2021 - 1.18               0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0   
GDP_2021 - 4.79              0.0  0.0  0.

## Unconstrained forecaster

The unconstrained forecaster generates forecasts using the DataFrame and forecaster. For DataFrames where all variables are unknown, the class essentially serves as a wrapper for the provided forecaster. 

In contrast, when exogenous variables are provided and .fit() is called, the class performs 3 key operations:

- Find and drop "island" constraints that follow NaNs (data.loc[2027, ('CA', 'A', 1)] in the example)
- Determine which variables are treated as known when (CA, TA are exogenous through 2023, while GDP and FX are exogenous for the entire sample)
- Generate forecasts with shifting exogenous variables

In our example, a different model will be estimated for IA between 2022-2023 and 2024-2030, where the former period will include CA and TA as exogenous while the latter will not.

In [9]:
print('\n\n Data: \n\n', data.tail(11))

from mff.step1_unreconciled_forecast import UnreconciledForecaster

fcaster = UnreconciledForecaster(data, forecaster)
fcaster.fit()
print('\n\n Data with deleted islands: \n\n', fcaster.df_no_islands)
print('\n\n Forecasts: \n\n', fcaster.y_hat)



 Data: 

 variable     CA    TA    IA    FX   GDP
freq          A     A     A     A     A
subperiod     1     1     1     1     1
year                                   
2020       0.55  2.11 -1.55  1.14 -3.34
2021      -2.46  0.03 -2.49  1.18  4.79
2022      -2.73 -0.27   NaN  1.05  4.51
2023      -2.98 -0.55   NaN  1.10  4.26
2024        NaN   NaN   NaN  1.10  4.04
2025        NaN   NaN   NaN  1.10  3.86
2026        NaN   NaN   NaN  1.10  3.72
2027      -0.30   NaN   NaN  1.10  3.60
2028        NaN   NaN   NaN  1.10  3.49
2029        NaN   NaN   NaN  1.10  3.39
2030        NaN   NaN   NaN  1.10  3.30


C:\ProgramData\Python3\Lib\site-packages\sktime\forecasting\compose\_reduce.py:1917: UserWarning: DirectReductionForecaster is experimental, and interfaces may change. user feedback is appreciated in issue #3224 here: https://github.com/sktime/sktime/issues/3224
  warn(
C:\ProgramData\Python3\Lib\site-packages\sktime\forecasting\compose\_reduce.py:1917: UserWarning: DirectReductionForecaster is experimental, and interfaces may change. user feedback is appreciated in issue #3224 here: https://github.com/sktime/sktime/issues/3224
  warn(
C:\ProgramData\Python3\Lib\site-packages\sktime\forecasting\compose\_reduce.py:1917: UserWarning: DirectReductionForecaster is experimental, and interfaces may change. user feedback is appreciated in issue #3224 here: https://github.com/sktime/sktime/issues/3224
  warn(
C:\ProgramData\Python3\Lib\site-packages\sktime\forecasting\compose\_reduce.py:1917: UserWarning: DirectReductionForecaster is experimental, and interfaces may change. user feedback is ap



 Data with deleted islands: 

 variable     CA     TA    IA    FX    GDP
freq          A      A     A     A      A
subperiod     1      1     1     1      1
year                                     
1994       4.19   4.51 -0.32  0.94   6.21
1995       2.69   2.38  0.30  1.01   7.87
1996      -9.67 -10.22  0.55  0.98   6.94
1997      -8.80  -8.99  0.19  0.90   4.44
1998      -9.57 -10.26  0.69  0.86   4.08
1999      -4.47  -4.12 -0.34  0.73  -0.11
2000      -3.08  -2.27 -0.81  0.65   1.17
2001      -7.98  -7.65 -0.33  0.62   3.25
2002      -7.60  -6.86 -0.74  0.67   4.51
2003      -4.89  -1.53 -3.36  0.82   5.50
2004      -5.85  -2.31 -3.54  0.94   5.28
2005      -7.29  -4.11 -3.17  0.97   6.62
2006      -7.03  -3.45 -3.57  1.02   8.49
2007      -4.80  -0.61 -4.19  1.22  10.83
2008      -6.36  -2.31 -4.05  1.42   5.57
2009      -3.44  -1.05 -2.40  1.39  -5.46
2010      -4.63  -1.06 -3.57  1.33   6.72
2011      -4.87  -0.43 -4.45  1.39   2.67
2012       0.93   3.97 -3.05  1.29   1.32
2

In addition to generating forecasts, the forecaster estimates an out-of-sample covariance matrix. This is done separately from the main .fit() call, as generating out-of-sample residuals for covariance matrices could be potentially computationally expensive. Additionally, to provide users the option to save on computation, the n_resids argument allows users to select the n_resid number of most recent observations to calculate the covariance matrix. 

The class implements three covariance estimators:
- Sample analog estimator
- Oracle shrinkage approximating estimator with diagonal target (OASD)- Monotone-increasing diagonal (replacing diagonal elements with the cumulative minimum value over horizons)

Additionally, users implement custom covariance calculations. This can be done by passing in a function that takes a DataFrame of residuals as an argument to "how".

The covariance matrix is accessed as a separate CovarianceMatrix class set as an attribute of forecaster. This allows users to easily switch the covariance calculation method by changing the .how property without reestimating residuals.

In [10]:
fcaster.fit_covariance(n_resid, how='raw')
print('\n\n Covariance matrix estimated with sample analog estimator: \n\n', fcaster.cov.cov_mat)

fcaster.cov.how = 'monotone_diagonal'
print('\n\n Covariance matrix estimated with monotone increasing diagonal: \n\n', fcaster.cov.cov_mat)

fcaster.cov.how = lambda x: x.cov()
print('\n\n Covariance matrix estimated with sample analog passed as a function: \n\n', fcaster.cov.cov_mat)

C:\ProgramData\Python3\Lib\site-packages\sktime\forecasting\compose\_reduce.py:1917: UserWarning: DirectReductionForecaster is experimental, and interfaces may change. user feedback is appreciated in issue #3224 here: https://github.com/sktime/sktime/issues/3224
  warn(
C:\ProgramData\Python3\Lib\site-packages\sktime\forecasting\compose\_reduce.py:1917: UserWarning: DirectReductionForecaster is experimental, and interfaces may change. user feedback is appreciated in issue #3224 here: https://github.com/sktime/sktime/issues/3224
  warn(
C:\ProgramData\Python3\Lib\site-packages\sktime\forecasting\compose\_reduce.py:1917: UserWarning: DirectReductionForecaster is experimental, and interfaces may change. user feedback is appreciated in issue #3224 here: https://github.com/sktime/sktime/issues/3224
  warn(
C:\ProgramData\Python3\Lib\site-packages\sktime\forecasting\compose\_reduce.py:1917: UserWarning: DirectReductionForecaster is experimental, and interfaces may change. user feedback is ap



 Covariance matrix estimated with sample analog estimator: 

 variable                            CA                                \
freq                                 A                                 
subperiod                            1                                 
year                              2024      2025      2026      2027   
variable freq subperiod year                                           
CA       A    1         2024  6.273566  1.721848 -0.887336  1.084253   
                        2025  1.721848  0.472580 -0.243539  0.297585   
                        2026 -0.887336 -0.243539  0.125505 -0.153357   
                        2027  1.084253  0.297585 -0.153357  0.187390   
                        2028  2.607396  0.715628 -0.368791  0.450633   
                        2029 -6.889475 -1.890891  0.974450 -1.190700   
                        2030  6.260613  1.718293 -0.885504  1.082014   
TA       A    1         2024  5.277313  1.448415 -0.746425  0.912072   


## Reconciler

The reconciler implements equation (10) in Ando (2024):

\begin{align}
\widetilde{y} = \left(I_{N} - (W^{-1} + \Phi)^{-1} C'(C(W^{-1} + \Phi)^{-1} C')\right) (W^{-1} + \Phi)^{-1} W^{-1} \widehat{y} + (W^{-1} + \Phi)^{-1}  C'(C(W^{-1} + \Phi)^{-1} C') d
\end{align}
where $W$ is the covariance matrix of unreconciled forecasts, $C$ and $d$ correspond to the constraints to ensure $C\widetilde{y} = d$, and $\Phi$ is the stacked matrix of HP filter terms (equation 8, Ando 2024).

In order to use this class, users must pass in:
- DataFrame of forecasts 
- DataFrame of exogenous variables
- Covariance matrix, W
- Constraints coefficient matrix, C
- Constraints constant matrix, d
- Regularization/smoothing parameter, $\lambda$

A dataframe of unreconciled forecasts containing indexes or columns with names ['variable', 'year', 'freq', 'subperiod']. They can then specify constraints in one of two ways. The first is to directly specify the matrix C and vector d. The second is to provide a DataFrame of exogenous forecasts to hold fixed. This dataframe is then transformed and appended to C and d vectors. 

When the matrix W is specified, the columns and rows must be a subset of the observations in the forecast DataFrame. However, not all  For variables where there is no entry provided, namely exogenous variables, the matrix W is padded with the identity matrix to ensure invertibility in equation (10). This does not alter the solution to the problem.
\begin{align}
W_{extended}^{-1} = \begin{bmatrix} W^{-1} & 0 \\ 0 & I_k
\end{bmatrix}
\end{align}

Lastly, users can specify how many horizons with which to pad the forecasts at the beginning and at the end to avoid endpoint issues related to the HP filter. By default, there is padding at the end by 2 observations (a number greater than 2 would not affect the optimization). To pad at the end, users can simply pass in a longer-than-desired horizon DataFrame in the forecasts.

The remainder of the estimation details are as in Ando (2024).

References:
Ando, Sakai (2024), “Smooth Forecast Reconciliation,” IMF Working Paper.

In [11]:
from mff.utils import load_forecast_data

df_forecast, W = load_forecast_data()

print(df_forecast, '\n\n\n')
print(W)

variable     CA     TA    IA    FX    GDP
freq          A      A     A     A      A
subperiod     1      1     1     1      1
year                                     
1994       4.19   4.51 -0.32  0.94   6.21
1995       2.69   2.38  0.30  1.01   7.87
1996      -9.67 -10.22  0.55  0.98   6.94
1997      -8.80  -8.99  0.19  0.90   4.44
1998      -9.57 -10.26  0.69  0.86   4.08
1999      -4.47  -4.12 -0.34  0.73  -0.11
2000      -3.08  -2.27 -0.81  0.65   1.17
2001      -7.98  -7.65 -0.33  0.62   3.25
2002      -7.60  -6.86 -0.74  0.67   4.51
2003      -4.89  -1.53 -3.36  0.82   5.50
2004      -5.85  -2.31 -3.54  0.94   5.28
2005      -7.29  -4.11 -3.17  0.97   6.62
2006      -7.03  -3.45 -3.57  1.02   8.49
2007      -4.80  -0.61 -4.19  1.22  10.83
2008      -6.36  -2.31 -4.05  1.42   5.57
2009      -3.44  -1.05 -2.40  1.39  -5.46
2010      -4.63  -1.06 -3.57  1.33   6.72
2011      -4.87  -0.43 -4.45  1.39   2.67
2012       0.93   3.97 -3.05  1.29   1.32
2013       1.85   4.55 -2.70  1.33

In [12]:
from mff.step2_reconciler import Reconciler

reconciler = Reconciler(df_forecast, data, W, C, d, lam)
reconciler.fit()

print('\n\n reconciler.y_reconciled: \n', reconciler.y_reconciled)
print('\n\n Reconciliation error: \n ', reconciler.y_reconciled['CA'] - reconciler.y_reconciled['IA'] - reconciler.y_reconciled['TA'])



 reconciler.y_reconciled: 
 variable          CA    FX   GDP        IA        TA
freq               A     A     A         A         A
subperiod          1     1     1         1         1
year                                                
2020        0.550000  1.14 -3.34 -1.550000  2.110000
2021       -2.460000  1.18  4.79 -2.490000  0.030000
2022       -2.730000  1.05  4.51 -2.460000 -0.270000
2023       -2.980000  1.10  4.26 -2.430001 -0.550000
2024        4.426379  1.10  4.04  2.986710  1.439669
2025       -0.815812  1.10  3.86 -3.928991  3.113179
2026       -3.470169  1.10  3.72 -8.736622  5.266453
2027       -0.300000  1.10  3.60 -3.858454  3.558454
2028        2.429645  1.10  3.49  3.573879 -1.144233
2029      -14.374479  1.10  3.39 -8.214757 -6.159722
2030        6.047216  1.10  3.30  1.121072  4.926145


 Reconciliation error: 
  freq                  A
subperiod             1
year                   
2020      -1.000000e-02
2021       2.498002e-16
2022      -2.873578e-07
202